In [ ]:
# Let's do some setup

# Install pytorch gpu

# Use the command below to install pytorch in gpu
#!pip3 install torch torchvision torchaudio

# Use command below to install pytorch in cpu
!pip3 install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cpu


Looking in indexes: https://download.pytorch.org/whl/cpu


In [ ]:
# unzip imagenette

In [ ]:
%cd /content/data
!ls
#!wget https://s3.amazonaws.com/fast-ai-imageclas/imagenette2.tgz

/content/data
class_code2name.json  class_id2name.json  imagenet_patch.gz
--2023-12-05 13:06:58--  https://s3.amazonaws.com/fast-ai-imageclas/imagenette2.tgz
Resolving s3.amazonaws.com (s3.amazonaws.com)... 52.216.216.224, 52.216.43.112, 52.216.61.144, ...
Connecting to s3.amazonaws.com (s3.amazonaws.com)|52.216.216.224|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1557161267 (1.5G) [application/x-tar]
Saving to: ‘imagenette2.tgz’

imagenette2.tgz     100%[===================>]   1.45G  23.3MB/s    in 56s     

2023-12-05 13:07:55 (26.4 MB/s) - ‘imagenette2.tgz’ saved [1557161267/1557161267]



In [ ]:
!python index.py --operation attack

program started for operation attack
building datasets...

Length of train dataset:  9296
Length of val dataset:  3856 

Reset output directories

Loading pretrained model

Attack images
	Adding patches to train batch 0
	Adding patches to train batch 1
	Adding patches to train batch 2
	Adding patches to train batch 3
	Adding patches to train batch 4
	Adding patches to train batch 5
	Adding patches to train batch 6
	Adding patches to train batch 7
	Adding patches to train batch 8
	Adding patches to train batch 9
	Adding patches to train batch 10
	Adding patches to train batch 11
	Adding patches to train batch 12
	Adding patches to train batch 13
	Adding patches to train batch 14
	Adding patches to train batch 15
	Adding patches to train batch 16
	Adding patches to train batch 17
	Adding patches to train batch 18
	Adding patches to train batch 19
	Adding patches to train batch 20


In [5]:
!python make_yolov5_dataset.py

No of images copied 699 618 81
['.config', '__init__.py', '.ipynb_checkpoints', 'models', 'transforms', 'test-mask-2.png', 'README.md', 'test_img.JPEG', 'datautils', 'utils', 'output', 'test-mask.png', 'make_yolov5_dataset.py', 'yolov5', 'preprocessor.py', 'train_loss_history.png', 'test-mask-3.png', 'attack', 'sample_patches.png', 'index.py', 'test-attacked-images', 'yolov5-attacked-images.zip', 'defence', 'config.py', 'data', 'runs', 'patch-detection.yaml', '__pycache__', 'attacked-images.zip', 'sample_data']
/content/data/yolov5-attacked-images /content/yolov5/data/yolov5-attacked-images


In [13]:
!python yolov5/train.py --img 224 --batch 32 --epochs 10 --data data/patch-detection.yaml --weights yolov5s.pt --workers 24 --name yolo-patch-detection-sm-final

[Errno 2] No such file or directory: 'yolov5/'
/content/yolov5
detect: weights=['runs/train/yolo-patch-detection-sm-final2/weights/best.pt'], source=data/yolov5-attacked-images/val/images, data=data/coco128.yaml, imgsz=[224, 224], conf_thres=0.25, iou_thres=0.45, max_det=1000, device=, view_img=False, save_txt=True, save_csv=False, save_conf=False, save_crop=False, nosave=False, classes=None, agnostic_nms=False, augment=False, visualize=False, update=False, project=runs/detect, name=exp, exist_ok=False, line_thickness=3, hide_labels=False, hide_conf=False, half=False, dnn=False, vid_stride=1
YOLOv5 🚀 v7.0-247-g3f02fde Python-3.10.12 torch-2.0.1+cu118 CUDA:0 (Tesla T4, 15102MiB)

Fusing layers... 
Model summary: 157 layers, 7015519 parameters, 0 gradients, 15.8 GFLOPs
image 1/199 /content/yolov5/data/yolov5-attacked-images/val/images/ILSVRC2012_val_00004650.JPEG: 224x224 1 patch, 5.1ms
image 2/199 /content/yolov5/data/yolov5-attacked-images/val/images/ILSVRC2012_val_00008161.JPEG: 224x2

In [ ]:
!python yolov5/detect.py --weights runs/train/yolo-patch-detection-sm-final2/weights/best.pt --img 224 --save-txt --source data/yolov5-attacked-images/val/images

In [ ]:
'''
%cd /content/drive/My\ Drive/UMass\ Projects/CS682/yolov5/data/patched-images/val/labels
!ls
import os
print(os.getcwd())
all_files = os.listdir(os.getcwd())
print(len(all_files))
for fn in all_files:
    data = []
    with open(fn, 'r', encoding='utf-8') as fp:
        data = fp.readlines()
    data[0] = data[0].replace(","," ")
    with open(fn, 'w', encoding='utf-8') as fp:
        fp.writelines(data)
'''

In [ ]:
#clean up yolo label
%cd /content/drive/My\ Drive/UMass\ Projects/CS682/yolov5
'''
import os
train_path = 'data/patched-images/train/labels'
valid_path = 'data/patched-images/val/labels'

train_fs = [fname for fname in os.listdir(train_path) if '.txt' in fname]
val_fs = [fname for fname in os.listdir(valid_path) if '.txt' in fname]

for fn in train_fs:
    fpath = os.path.join(train_path, fn)
    data = ''
    with open(fpath, 'r', encoding='utf-8') as fp:
        data = fp.readlines()[0]
    if "," in data:
        data = data.split(",")
        xc, yc, w, h = int(data[1]), int(data[2]), int(data[3]), int(data[4])
        xc = ((xc + 50) - xc / 2)/224
        yc = ((yc + 50) - yc / 2)/224
        data[1] = xc
        data[2] = yc
        data[3] = w / 224
        data[4] = h / 224
        new_ln = f'{data[0]} {data[1]} {data[2]} {data[3]} {data[4]}'

        with open(fpath, 'w+', encoding='utf-8') as fp:
            fp.writelines(new_ln)

for fn in val_fs:
    fpath = os.path.join(valid_path, fn)
    data = ''
    with open(fpath, 'r', encoding='utf-8') as fp:
        data = fp.readlines()[0]
    if "," in data:
        data = data.split(",")
        xc, yc, w, h = int(data[1]), int(data[2]), int(data[3]), int(data[4])
        xc = ((xc + 50) - xc / 2)/224
        yc = ((yc + 50) - yc / 2)/224
        data[1] = xc
        data[2] = yc
        data[3] = w / 224
        data[4] = h / 224
        new_ln = f'{data[0]} {data[1]} {data[2]} {data[3]} {data[4]}'

        with open(fpath, 'w+', encoding='utf-8') as fp:
            fp.writelines(new_ln)
'''